In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction to Word2Vec NLP
[This](https://arxiv.org/pdf/1301.3781.pdf) is the original paper published by Google Research

<center><img src="imgs/wv0.webp" width=50%></center>


We need to create a vocabulary of all the words in our text and then to encode our word as a vector of the same dimensions of our vocabulary, so this is exactly using `OneHotEncoding`, this input is given to a neural network with a single hidden layer.

<center><img src="imgs/wv1.webp" width=30%></center>

The output of such network is a **single vector (also with the same length components) containing** that  represents the **probability** that a randomly selected nearby word is that vocabulary word.

In word2vec, a distributed representation of a word is used. Take a vector with several hundred dimensions (say 1000). Each word is represented by a distribution of weights across those elements. So instead of a one-to-one mapping between an element in the vector and a word, **the representation of a word is spread across all the elements in the vector**, and **each element in the vector contributes to the definition of many words**. Such a vector comes to represent in some abstract way the ‘meaning’ of a word.
- It is not one word that represenst one word but rather "all" words that represents one word
- This is the distributional semantics idea, using an distribution of an sentence to encode/embed the meaning of an word


<center><img src="imgs/wv2.webp" width=50%></center>

## CBOW (Continuous Bag of Words) & Continuous Skip-Grams
So that is the basic Idea, how does it achieve such magical embedding?

In the `CBOW model`, the **distributed representations of context** (or surrounding words) are combined to **predict the word in the middle**. While in the `Skip-gram model`, the **distributed representation of the input word** is used to predict the **context**.

<center><img src="imgs/wv3.png" width=70%></center>

### In `CBOW`
- Note that `CBOW` does not consider orders of the word, it takes in both "past" words and "future" words, just like in regular `bag of words` model. However, it uses continuous distributed representation of the context.

- In `CBOW`, since our input vectors are `OneHotEncoding`, multiplying an input vector by the weight matrix `W1` is just selecting a row/word from `W1`. From the hidden layer to the output layer, the second weight matrix `W2` can be used to compute **a score** for each word in the vocabulary, and **softmax** can be used to obtain the **posterior distribution of words**.

### In `Skip-Gram`
- The `skip-gram model` is the opposite of the `CBOW` model. It is constructed with the **focus word as the single input vector**, and the target context words are now at the output layer. The activation function for the hidden layer simply amounts to copying the corresponding row from the weights matrix `W1` (linear) as we saw before. At the output layer, we now output **C multinomial distributions instead of just one**.

- Given the sentence: *“I will have orange juice and eggs for breakfast.”*
    - and a window size of 2, if the target word is juice, its neighboring words will be ( have, orange, and, eggs). Our input and target word pair would be (juice, have), (juice, orange), (juice, and), (juice, eggs).

    - Also note that within the sample window, proximity of the words to the source word plays no role. So have, orange, and, and eggs will be treated the same while training.

    - The dimensions of the input vector will be **1xV** — where V is the number of words in the vocabulary — i.e `OneHotEncoding` representation of the word. The single hidden layer will have dimension **VxE**, where E is the size of the word embedding and is a hyper-parameter. The output from the hidden layer would be of the dimension **1xE**, which we will feed into an `softmax` layer. The dimensions of the output layer will be 1xV, where each value in the vector will be *the probability score of the target word at that position*.

<center><img src="imgs/window.webp" width=50%></center>

## Implementation Details

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
pd.options.plotting.backend = 'plotly'
from itertools import chain

from utils.eda import *
from utils.dsc80_utils import *
from utils.graph import *
from gensim.models import Word2Vec
import nltk
# nltk.download('brown')
# nltk.download('movie_reviews')
# nltk.download('treebank')
# from nltk.corpus import brown, movie_reviews, treebank
from nltk.tokenize import word_tokenize

In [7]:
interactions = pd.read_csv('food_data/RAW_interactions.csv')
recipes = pd.read_csv('food_data/RAW_recipes.csv')
step0 = recipes.merge(interactions, how='left', left_on='id', right_on='recipe_id', indicator=True)
df_recipe = (step0
      .pipe(initial)
      .pipe(transform_df)
      #.pipe(outlier)
      .pipe(group_recipe)
      #.pipe(group_user)
)

In [8]:
# b = Word2Vec(brown.sents())
# mr = Word2Vec(movie_reviews.sents())
# t = Word2Vec(treebank.sents())
# brown.sents()
# b.wv.most_similar('food', topn=5)

In [9]:
# corpus = ' '.join(df['description'].astype(str))
# tokens = word_tokenize(corpus)
# tokens
tokens = df_recipe['description'].astype(str).str.split(' ').to_list()
tokens[0]

['one',
 'of',
 'my',
 "mom's",
 'favorite',
 'bisquick',
 'recipes.',
 'this',
 'brings',
 'back',
 'memories!']

In [10]:
word_vec = Word2Vec(tokens, window=7, sg=1, min_count=3) # input is a list of list

In [18]:
word_vec.wv.most_similar('cheese', topn=10)

[('cheddar', 0.7908340692520142),
 ('goat', 0.7435830235481262),
 ('fontina', 0.737315833568573),
 ('goats', 0.731482982635498),
 ('parmesan', 0.7294240593910217),
 ('gruyere', 0.7245720624923706),
 ('montrachet', 0.722097635269165),
 ('romano', 0.7213541865348816),
 ('tillamook', 0.7174773812294006),
 ('havarti', 0.7120335102081299)]

In [12]:
df_user = (step0
      .pipe(initial)
      .pipe(transform_df)
      #.pipe(outlier)
      #.pipe(group_recipe)
      .pipe(group_user)
)

/Users/kevinb/Desktop/dsc80/dsc80_proj/utils/eda.py:115: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [13]:
user_corpus = df_user['description'].astype(str).str.strip('[]').str.strip("'").str.strip('"').str.split(' ')
user_corpus.shape

(67268,)

In [14]:
word_vec = Word2Vec(user_corpus, window=7, sg=1, min_count=3)

In [17]:
word_vec.wv.most_similar('cheese', topn=10)

[('cheddar', 0.7908340692520142),
 ('goat', 0.7435830235481262),
 ('fontina', 0.737315833568573),
 ('goats', 0.731482982635498),
 ('parmesan', 0.7294240593910217),
 ('gruyere', 0.7245720624923706),
 ('montrachet', 0.722097635269165),
 ('romano', 0.7213541865348816),
 ('tillamook', 0.7174773812294006),
 ('havarti', 0.7120335102081299)]

We can use the `distance` function

In [16]:
word_vec.wv.distance('food','delicious')

0.7651446610689163